# Actor Critic pour résoudre l'environnement OpenAI Gym Cartpole

Copie expliquée de la page https://keras.io/examples/rl/actor_critic_cartpole/

## Introduction 

L'algorithme Actor Critic est une famille d'algorithme de Reinforcement Learning qui divise la tâche de sélection de l'action en deux parties à savoir :
- une partie (Critic) qui évalue l'intérêt ou la valeur de l'action au regard de l'environnement et du système considéré
- une partie (Actor) qui en utilisant les informations générées par le Critic choisit effectivement l'action pour l'état présent



***
On commence par importer les packages nécessaires au programme et on met en place l'environnement qui sera utilisé par l'algorithme. On définit ensuite quelques variables qui seront utiles au problème. 

In [2]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Configuration parameters for the whole setup
seed = 42
gamma = 0.99  # Discount factor for past rewards
max_steps_per_episode = 10000
env = gym.make("CartPole-v0")  # Create the environment
env.seed(seed)
eps = np.finfo(np.float32).eps.item()  # Smallest number such that 1.0 + eps != 1.0

In [2]:
num_inputs = 4
num_actions = 2
num_hidden = 128

inputs = layers.Input(shape=(num_inputs,))
common = layers.Dense(num_hidden, activation = 'relu')(inputs)
action = layers.Dense(num_actions, activation = "softmax")(common)
critic = layers.Dense(1)(common)

model = keras.Model(inputs = inputs, outputs = [action, critic])

# actor and critic tend to share initial layer weight to fasten convergence

In [3]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
huber_loss = keras.losses.Huber()
action_probs_history=[]
critic_value_history=[]
reward_history=[]
running_reward=0
episode_count=0

while True:
    state = env.reset()
    episode_reward=0
    with tf.GradientTape() as tape: # A retravailler
        for timestep in range(1, max_steps_per_episode):
            # env.render() # Si on veux le rendu
            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)
            
            # pour l'état présent, renvoie les sorties des deux 2N Actor et Critic
            # stocke l'avantage pour des calculs ultérieurs hors de la boucle
            action_probs, critic_value = model(state)
            critic_value_history.append(critic_value[0,0])
            # print(action_probs)
            # print(critic_value)
            # Policy : donne l'action selon les actions possibles et la distribution d'actions donnée par l'actor
            # puis stocke cette action dans l'historique
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            action_probs_history.append(tf.math.log(action_probs[0,action]))
            
            # Joue l'environnement pour l'état présent et l'action précédemment choisie
            # puis stocke la récompense dans l'historique
            # Si done, alors le pendule est tombé et la simulation n'est plus intéressante.
            state, reward, done, _ = env.step(action)
            reward_history.append(reward)
            episode_reward += reward
            
            if done:
                break 
        
        running_reward = 0.05*episode_reward + (1-0.05)*running_reward
        
        # Calcul du retour attendu pour l'état présent 
        # le calcul 
        
        returns = []
        discounted_sum=0
        for r in reward_history[::-1]:
            discounted_sum = r + gamma*discounted_sum
            returns.insert(0, discounted_sum)
        returns = np.array(returns)
        returns = (returns - np.mean(returns))/(np.std(returns)+eps)
        returns = returns.tolist()
        
        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for log_prob, value, ret in history:
            adv = ret - value 
            actor_losses.append(-log_prob * adv)
            critic_losses.append(huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret,0))  )
            
        loss_value = sum(actor_losses) + sum(critic_losses)
        # print(loss_value)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        action_probs_history.clear()
        critic_value_history.clear()
        reward_history.clear()
        
        episode_count += 1
        if episode_count%10==0:
            print("Running reward {} at episode {}".format(running_reward, episode_count))
        
        if running_reward > 195: # Condition to consider the task solved 
            print("Solved at episode {}!".format(episode_count))
            break


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Running reward 13.238667661672265 at episode 10
Running reward 27.384802149544235 at episode 20
Running reward 30.398359697927035 at episode 30
Running reward 34.4238415627547 at episode 40
Running reward 31.575467282025855 at episode 50
Running reward 31.938062653795036 at episode 60
Running reward 43.2134828329286 at episode 70
Running reward 43.576382205728244 at episode 80
Running reward 33.268026518601395 at episode 90
Running reward 29.718090561352323 at episode 100
Running reward 29.5655696971737

In [4]:
model.save('A2C')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: A2C\assets


In [1]:
reward_history=[]

while True:
    state = env.reset()
    # episode_reward=0
    with tf.GradientTape() as tape: # A retravailler
        for timestep in range(1, max_steps_per_episode):
            env.render() # Si on veux le rendu
            state = tf.convert_to_tensor(state)
            state = tf.expand_dims(state, 0)
            
            # pour l'état présent, renvoie les sorties des deux 2N Actor et Critic
            # stocke l'avantage pour des calculs ultérieurs hors de la boucle
            action_probs, critic_value = model(state)
            # critic_value_history.append(critic_value[0,0])
            # print(action_probs)
            # print(critic_value)
            # Policy : donne l'action selon les actions possibles et la distribution d'actions donnée par l'actor
            # puis stocke cette action dans l'historique
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            # action_probs_history.append(tf.math.log(action_probs[0,action]))
            
            # Joue l'environnement pour l'état présent et l'action précédemment choisie
            # puis stocke la récompense dans l'historique
            # Si done, alors le pendule est tombé et la simulation n'est plus intéressante.
            state, reward, done, _ = env.step(action)
            reward_history.append(reward)
            # episode_reward += reward
            
            # if done:
                # break 

NameError: name 'env' is not defined